### Setup

In [2]:
import os, sys, importlib, time
from dotenv import load_dotenv

# Get directories first to find .env file
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
env_path = os.path.join(parent_dir, '.env')

# Load .env file
load_dotenv(env_path, override=True)

# Get timezone from .env with default fallback
timezone = os.getenv('TZ', 'America/New_York')
os.environ["TZ"] = timezone
time.tzset()

# Get additional directories
rag_dir = os.path.join(parent_dir, 'RAG')
print(f"Current directory: {current_dir}")
print(f"Parent directory: {parent_dir}")
print(f"Parent dir exists: {os.path.exists(parent_dir)}")
print(f"RAG dir exists: {os.path.exists(rag_dir)}")
print(f"KnowledgeGraph exists: {os.path.exists(os.path.join(parent_dir, 'KnowledgeGraph'))}")

# Add paths to sys.path BEFORE importing modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
if rag_dir not in sys.path:
    sys.path.insert(0, rag_dir)
    
# Import modules after paths are set up
from KnowledgeGraph.config import load_neo4j_graph
import RAG.VectorRAG as VectorRAG
import RAG.GraphRAG as GraphRAG
import RAG.batch_query as batch_query

# make sure AB Testing is on sys.path
test_dir = os.path.join(parent_dir, "AB Testing")
if test_dir not in sys.path:
    sys.path.insert(0, test_dir)

# import/reload mark_answers, then get the function
import mark_answers as mark_answers_module
importlib.reload(mark_answers_module)
mark_answers = mark_answers_module.mark_answers

# Reload other modules to pick up changes
importlib.reload(VectorRAG)
importlib.reload(GraphRAG)
importlib.reload(batch_query)

# TODO Remove this
from RAG.GraphRAG import query_graph_rag

# TODO understand this
graph, openAI_api, openAI_endpoint, openAI_model = load_neo4j_graph()

Current directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/Main Functions
Parent directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3
Parent dir exists: True
RAG dir exists: True
KnowledgeGraph exists: True


/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j/_sync/driver.py:547: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(


### GraphRag Query

In [5]:
GraphRAG.query_graph_rag("What is the closing date for Kuwait Oil Company Tender No. 2125611-RFP?")

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/retrievers/vector.py:201: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_params = get_search_query(


'The closing date for Kuwait Oil Company Tender No. 2125611-RFP is not provided in the supplied context. The excerpt lists other tenders (RFQ/1056222, RFQ/1056372, RFQ/1056315) but not 2125611-RFP. If you can share the specific announcement for 2125611-RFP, I can extract the closing date.'

In [6]:
batch_query.batch_query_graph_rag("AB Testing/questions_long.json", max_workers=60)

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/gen

Processed 71 queries with 60 workers
Results saved to: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_042856.json


[{'question': 'What is the time period to submit an objection for company registration changes?',
  'answer': 'It depends on the type of change announced:\n\n- 30 days from the date of publication to file an objection with the General Court for legal‑entity amendments (e.g., LLC ↔ One‑Person Company, partnership → LLC) and for dissolution/liquidation notices.\n- 15 days from the date of publication to submit a written objection (with a debt instrument) to the Companies Department for converting an individual license/establishment into a company (LLC or One‑Person Company).'},
 {'question': 'Where should objections to company registration be submitted?',
  'answer': 'The General Court (Court of First Instance, المحكمة الكلية). Note: For some license-conversion notices, objections must instead be filed in writing with the Companies Department within 15 days, accompanied by a debt instrument.'},
 {'question': 'What types of legal entities are mentioned in company registration announcement

### Mark Answers

In [8]:
mark_answers(answer_key_path = "AB Testing/answer_key_long.json", max_workers = 60)


TEST RESULTS

Configuration:
  top_k: 30
  MMR_k: 15
  MMR_LAMBDA: 0.4
  ALWAYS_KEEP_TOP: 3
  neighbor_window: 1

Results:
  Average F1: 48.4% (0.484)

Card-first gate:
  CARD_FIRST_ENABLED: True
  ALLOW_DIRECT_ANSWER: True



{'saved_at': '2025-11-02T04:30:37',
 'expected_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/answer_key_long.json',
 'answers_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_042856.json',
 'config': {'top_k': 30,
  'MMR_k': 15,
  'MMR_LAMBDA': 0.4,
  'ALWAYS_KEEP_TOP': 3,
  'neighbor_window': 1,
  'CARD_FIRST_ENABLED': True,
  'ALLOW_DIRECT_ANSWER': True},
 'overall': {'evaluated': 71.0,
  'average_f1': 0.4844366197183099,
  'average_f1_percent': 48.44366197183099},
 'questions': [{'question': 'What is the time period to submit an objection for company registration changes?',
   'expected': 'Thirty days from the date of publication in the official gazette [1].\n\nReferences\n1. Kuwait Official Gazette 2025-04-27, Lines 5228-5327.',
   'answer': 'It depends on the type of change announced:\n\n- 30 days from the date of publication to file an objection with the General 